In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
from mpl_toolkits.basemap import Basemap
import cartopy.crs as ccrs
import cartopy.feature as cfeature
AOD=[]
import xarray as xr
from netCDF4 import Dataset
import glob
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
from mpl_toolkits.basemap import Basemap
import cartopy.crs as ccrs
import cartopy.feature as cfeature
AOD=[]
import xarray as xr
os.chdir("/home/ybh10/Scripts/")
from my_functions import *
os.chdir("/home/ybh10/DMS_Emissions/Numpy_Array/")
# Daily_Stash=load_dict('Daily_Stash_Output.npy')
# Monthly_Stash=load_dict('Monthly_Stash_Output.npy')
MODIS_Clim = np.load('AOD_MODIS_DATA.npy')


os.chdir("/nesi/nobackup/niwa02757/ybh10/DMS")
inpath_lana=sorted(glob.glob('/2003/*.nc'))
inpath_medusa=sorted(glob.glob('MEDUSA/2003/*.nc'))
months = ['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']
vn1=xr.open_dataset('Lana/2003/bx_atmosphere_optical_thickness_due_to_dust_ambient_aerosol_2003_jan.nc')
lat=vn1.latitude
lon=vn1.longitude
Dates=['January', 'February','March','April','May','June','July','August','September','October','November','December']
lat_bnds = [-60, -40]
lat_inds = np.where((lat > lat_bnds[0]) & (lat < lat_bnds[1]))
lat_inds = np.squeeze(lat_inds)
lat_sh=vn1.latitude[lat_inds[0]:lat_inds[(len(lat_inds)-1)]+1]

inpath_modis=sorted(glob.glob('/nesi/nobackup/niwa02757/ybh10/Observational_Data/MODIS/AOD/*.nc'))
latty_o=xr.open_dataset(inpath_modis[0])
lat_o=latty_o['lat']
lon_o=latty_o['lon']
lat_modis_sh = np.load('/home/ybh10/DMS_Emissions/Numpy_Array/lat_modis_SH.npy')
modis = np.load('/home/ybh10/DMS_Emissions/Numpy_Array/AOD_MODIS_DATA.npy')
modis_sh = np.load('/home/ybh10/DMS_Emissions/Numpy_Array/AOD_MODIS_SH_DATA.npy')

In [2]:

file='/home/ybh10/DMS_Emissions/Chemistry_Scheme/CHEM2.1/Daily/'

In [11]:
def interpolate(Observational_data,Observational_lat, Observational_lon, model_lat,model_lon,model_data):
    mask_test = np.isnan(Observational_data)
    Aerosol=check_lat_lon(model_lat,model_lon,model_data)
    Aerosol_model=Aerosol[2]
    lats_model=Aerosol[0]
    lons_model=Aerosol[1]
    obs=check_lat_lon(Observational_lat,Observational_lon,Observational_data)
    Aerosol_obs=obs[2]
    lats_obs=obs[0]
    lons_obs=obs[1]
    AOD_M_Modified=my_interpolate(lats_model,lons_model,Aerosol_model,Observational_lat,Observational_lon)
 #   AOD_M_Modified[mask_test] = np.nan
    return AOD_M_Modified,mask_test

In [ ]:
### SAVES AOD ###\
aod_variables=['atmosphere_optical_thickness_due_to_dust_ambient_aerosol','atmosphere_optical_thickness_due_to_insoluble_aitken_mode_sulphate_aerosol',
'atmosphere_optical_thickness_due_to_soluble_accumulation_mode_sulphate_aerosol','atmosphere_optical_thickness_due_to_soluble_aitken_mode_sulphate_aerosol',
'atmosphere_optical_thickness_due_to_soluble_coarse_mode_sulphate_aerosol']
lana_aod=[]  # TAKES all AOD files of each month, sums it to make the AOD for each month. then stores this data as AOD.
lana_aod_sh=[]
for mon in (months):
    files=sorted(glob.glob(file+'*d_atmosphere_optical_thickness_due_to**{}*.nc'.format(mon)))
    aerosol=0
    aerosol_sh=0
    for x,vari in zip(files,aod_variables):
        lat_inds = np.where((lat > lat_bnds[0]) & (lat < lat_bnds[1]))
        lat_inds = np.squeeze(lat_inds)

        vn1=xr.open_dataset(x)
        
     #   print(x)
        num=vn1[vari][2].data
        num_sh=vn1[vari][2,:,lat_inds[0]:lat_inds[(len(lat_inds)-1)]+1,:].data
        aerosol=aerosol+num
        aerosol_sh=aerosol_sh+num_sh
    #print('------------------')
    model_sh_aerosol=np.nanmean(aerosol_sh,axis=0)
    model_aerosol=np.nanmean(aerosol,axis=0)
    aerosol_check=check_lat_lon(lat,lon,aerosol)
    obs_check=check_lat_lon(lat_o,lon_o,MODIS_Clim)

    data=interpolate(MODIS_Clim,lat_o,lon_o,lat,lon,aerosol)
    data_sh=interpolate(modis_sh,lat_modis_sh,lon_o,lat_sh,lon,aerosol_sh)
    #my_interpolate(lat,aerosol_check[1],aerosol_check[2],lat_o,lon_o)
    lana_aod.append(np.nanmean(data[0],axis=0))
    lana_aod_sh.append(np.nanmean(data_sh[0],axis=0))

    
lana_aod_sh=np.array(lana_aod_sh)
lana_aod=np.array(lana_aod)

lana_aod[data[1]] = np.nan
lana_aod_sh[data_sh[1]] = np.nan

In [12]:
lana_dms=[]  # DMS
lana_dms_sh=[]
#files=sorted(glob.glob(file+'*d_DMS_surface_emissions_**{}*.nc'.format(mon)))

for mon in (months):
    files=sorted(glob.glob(file+'*d_DMS_surface_emissions_**{}*.nc'.format(mon)))
    print(files)
    vn1=xr.open_dataset(files[0])
    num=vn1['unknown'].data
    num_sh=vn1['unknown'][:,lat_inds[0]:lat_inds[(len(lat_inds)-1)]+1,:].data
    
    aerosol_check=check_lat_lon(lat,lon,num)
    obs_check=check_lat_lon(lat_o,lon_o,MODIS_Clim)
    
    data=interpolate(MODIS_Clim,lat_o,lon_o,lat,lon,num)
    data_sh=interpolate(modis_sh,lat_modis_sh,lon_o,lat_sh,lon,num_sh)

    lana_dms.append(np.nanmean(data[0],axis=0))
    lana_dms_sh.append(np.nanmean(data_sh[0],axis=0))

    
lana_dms=np.array(lana_dms)
lana_dms_sh=np.array(lana_dms_sh)

lana_dms[data[1]] = np.nan
lana_dms_sh[data_sh[1]] = np.nan

['/home/ybh10/DMS_Emissions/Chemistry_Scheme/CHEM2.1/Daily/d_DMS_surface_emissions_1989jan.nc']
['/home/ybh10/DMS_Emissions/Chemistry_Scheme/CHEM2.1/Daily/d_DMS_surface_emissions_1989feb.nc']
['/home/ybh10/DMS_Emissions/Chemistry_Scheme/CHEM2.1/Daily/d_DMS_surface_emissions_1989mar.nc']


KeyboardInterrupt: 

In [41]:
lana_msa=[]  # MSA
lana_msa_sh=[] 
#files=sorted(glob.glob(file+'*d_DMS_surface_emissions_**{}*.nc'.format(mon)))

for mon in (months):
    files=sorted(glob.glob(file+'*d_MSA_Mass_Mixing_Ratio_**{}*.nc'.format(mon)))
    print(files)
    vn1=xr.open_dataset(files[0])
    num=vn1['mass_fraction_of_methanesulfonic_acid_in_air'][:,0].data
    num_sh=vn1['mass_fraction_of_methanesulfonic_acid_in_air'][:,0,lat_inds[0]:lat_inds[(len(lat_inds)-1)]+1,:].data
    
    aerosol_check=check_lat_lon(lat,lon,num)
    obs_check=check_lat_lon(lat_o,lon_o,MODIS_Clim)
    
    data=interpolate(MODIS_Clim,lat_o,lon_o,lat,lon,num)
    data_sh=interpolate(modis_sh,lat_modis_sh,lon_o,lat_sh,lon,num_sh)
    print(mon)
    lana_msa.append(np.nanmean(data[0],axis=0))
    lana_msa_sh.append(np.nanmean(data_sh[0],axis=0))

    
lana_dms=np.array(lana_msa)
lana_dms_sh=np.array(lana_msa_sh)

# lana_dms[data[1]] = np.nan
# lana_dms_sh[data_sh[1]] = np.nan

['/home/ybh10/DMS_Emissions/Chemistry_Scheme/CHEM2.1/Daily/d_MSA_Mass_Mixing_Ratio_1989jan.nc']
jan
['/home/ybh10/DMS_Emissions/Chemistry_Scheme/CHEM2.1/Daily/d_MSA_Mass_Mixing_Ratio_1989feb.nc']
feb
['/home/ybh10/DMS_Emissions/Chemistry_Scheme/CHEM2.1/Daily/d_MSA_Mass_Mixing_Ratio_1989mar.nc']
mar
['/home/ybh10/DMS_Emissions/Chemistry_Scheme/CHEM2.1/Daily/d_MSA_Mass_Mixing_Ratio_1989apr.nc']
apr
['/home/ybh10/DMS_Emissions/Chemistry_Scheme/CHEM2.1/Daily/d_MSA_Mass_Mixing_Ratio_1989may.nc']
may
['/home/ybh10/DMS_Emissions/Chemistry_Scheme/CHEM2.1/Daily/d_MSA_Mass_Mixing_Ratio_1989jun.nc']
jun
['/home/ybh10/DMS_Emissions/Chemistry_Scheme/CHEM2.1/Daily/d_MSA_Mass_Mixing_Ratio_1989jul.nc']
jul
['/home/ybh10/DMS_Emissions/Chemistry_Scheme/CHEM2.1/Daily/d_MSA_Mass_Mixing_Ratio_1989aug.nc']
aug
['/home/ybh10/DMS_Emissions/Chemistry_Scheme/CHEM2.1/Daily/d_MSA_Mass_Mixing_Ratio_1988sep.nc']
sep
['/home/ybh10/DMS_Emissions/Chemistry_Scheme/CHEM2.1/Daily/d_MSA_Mass_Mixing_Ratio_1988oct.nc']
oct
